## **Python OER**
### Quick Crosswalk Cases - BMI
### Python Solution
#### **Author(s)**: Dominic DiSanto
#### **Written**: 12/12/2020
#### **Updated**: 12/12/2020
#### **Version**: 0.1


Recalling the prompt for this use case, we need to identify the contact information for each patient that meets the following criteria:

1) BMI $\geq$ 30  
2) BMI $\geq$ 35  
3) BMI $\geq$ 30 & Age$\geq$60  
4) BMI $\geq$ 35 & Age$\geq$60  

We will import the two tabs of our spreadsheet as two separate dataframes. THen, given that we only have height and weight data, in our original "master" data set we must first calculate BMI in our original data frame of height & weight data.

In [1]:
import pandas as pd
import numpy as np 

In [2]:
bmi_df = pd.read_excel('BMI_Data.xlsx', sheet_name = 'HeightWeight')
bmi_df.head()

,ID,Age,Height (cm),Weight (kg)
0,458439593,62,220.1,196.20
1,466502951,50,192.0,100.64
2,780243050,62,209.8,193.67
3,197598223,28,185.9,103.33
4,918135592,67,148.4,62.10


In [3]:
contact_df = pd.read_excel('BMI_Data.xlsx', sheet_name = 'Contact Info')
contact_df.head()

,ID,PhoneNo,Address
0,458439593,001-766-098-1571,"22326 Jensen Mountains Suite 987\nJonesmouth, ..."
1,466502951,635.237.2550,"961 Jennifer Pike Suite 707\nSouth Donmouth, G..."
2,780243050,828.528.2249x47820,"23551 Mahoney Junction\nWest Brandon, MI 53433"
3,197598223,0080358260,"741 Sanchez Stravenue Suite 840\nMichellestad,..."
4,918135592,6632740778,"581 Holland Cove\nCoryburgh, KS 17977"


In [4]:
# Calculating BMI (remember to round and to re-calculate height in meters)
bmi_df['BMI'] = np.round(bmi_df['Weight (kg)'] / (bmi_df['Height (cm)']/100)**2, 2)
bmi_df.head()

,ID,Age,Height (cm),Weight (kg),BMI
0,458439593,62,220.1,196.20,40.5
1,466502951,50,192.0,100.64,27.3
2,780243050,62,209.8,193.67,44.0
3,197598223,28,185.9,103.33,29.9
4,918135592,67,148.4,62.10,28.2


Now that we've calculated BMI, we want to identify the four subsets of interest in our cohort and merge in the contact information for each set. We will save each data frame, and export all relevant data into an excel spreadsheet in a later code chunk.

In [5]:
# (1) BMI>=30
subset1 = pd.merge(bmi_df.loc[bmi_df['BMI']>=30, 'ID'], contact_df, on='ID')
subset1 = subset1.sort_values('ID')

# (2) BMI>=35
subset2 = pd.merge(bmi_df.loc[bmi_df['BMI']>=35, 'ID'], contact_df, on='ID')
subset2 = subset2.sort_values('ID')

# (3) BMI>=30 & Age>=60
subset3 = pd.merge(bmi_df.loc[(bmi_df['BMI']>=30) & (bmi_df['Age']>=60), 'ID'], contact_df, on='ID')
subset3 = subset3.sort_values('ID')

# (4) BMI>=35 & Age>=60
subset4 = pd.merge(bmi_df.loc[(bmi_df['BMI']>=35) & (bmi_df['Age']>=60), 'ID'], contact_df, on='ID')
subset4 = subset4.sort_values('ID')

Now that we have identified our four cohorts of interest, we want to export them all within the same `xlsx` file but simply as different tabs. We can conveniently use pandas's ExcelWriter() to specify and export data frames as tables within a single file.

In [6]:
with pd.ExcelWriter('BMI_Solution_Python.xlsx') as writer:
    subset1.to_excel(writer,
                    'Criteria_1', index=False)
    subset2.to_excel(writer,
                'Criteria_2', index=False)
    subset3.to_excel(writer,
                'Criteria_3', index=False)
    subset4.to_excel(writer,
                'Criteria_4', index=False)
        

### Data Update

Having received our "updated data", we can use Python to update the patients & contact information of interest using the same criteria. We could simply import the updated data and use the above code verbatim by importing the updated data into the same `bmi_df` and `contact_df` objects, or   
`contact_df = pd.read_excel('BMI_Data.xlsx', sheet_name = 'Contact Info')`  
and  
`bmi_df = pd.read_excel('BMI_Data.xlsx', sheet_name = 'HeightWeight')`  
and changing the export file name in our ExcelWriter() statement as 
`with pd.ExcelWriter('BMI_Solution_Update.xlsx) as writer:`   
etc.

This is a perfectly acceptable solution! However, if we know this will be an interative process, with not one but multiple and possibly even many updates required, we could define a function that allows us to simply output the results in an update with only one line (assuming the same criteria). The below code includes an example of creating and then running function that will create our subsets and export a spreadsheet. 

A last note, simply recycling your above code (i.e. not creating a function as we do below) is again a perfectly acceptable answer for this use case! The below code simply provides this as some "extra" code to show you another solution that is useful for other iterative processes. 

In [7]:
# Supplying default values for a number of parameters but also including some flexibility
# by allowing for different cutoffs or variable names in the initial data 
def contact_pull(file, bmi_sheet, contact_sheet, output_file,
                 age_cutoff = 60, bmi_cutoff1 = 30, bmi_cutoff2 = 35,
                bmi_var = 'BMI', id_var = 'ID', age_var = 'Age', 
                weight_var = 'Weight (kg)', height_var = 'Height (cm)'):
    
    bmi_df = pd.read_excel(file, sheet_name = bmi_sheet)
    bmi_df[bmi_var] = np.round(bmi_df[weight_var] / (bmi_df[height_var]/100)**2, 2)

    contact_df = pd.read_excel(file, sheet_name = contact_sheet)
    
    # (1) BMI>=30
    subset1 = pd.merge(bmi_df.loc[bmi_df[bmi_var]>=bmi_cutoff1, id_var], contact_df, on=id_var)
    subset1 = subset1.sort_values(id_var)

    # (2) BMI>=35
    subset2 = pd.merge(bmi_df.loc[bmi_df[bmi_var]>=bmi_cutoff2, id_var], contact_df, on=id_var)
    subset2 = subset2.sort_values(id_var)

    # (3) BMI>=30 & Age>=60
    subset3 = pd.merge(bmi_df.loc[(bmi_df[bmi_var]>=bmi_cutoff1) & (bmi_df[age_var]>=age_cutoff),id_var],
                       contact_df, on=id_var)
    subset3 = subset3.sort_values(id_var)

    # (4) BMI>=35 & Age>=60
    subset4 = pd.merge(bmi_df.loc[(bmi_df[bmi_var]>=bmi_cutoff2) & (bmi_df[age_var]>=age_cutoff), id_var],
                       contact_df, on=id_var)
    subset4 = subset4.sort_values(id_var)
    
    if output_file[-5:len(output_file)] != '.xlsx':
        output_file = output_file + '.xlsx'
        
    with pd.ExcelWriter(output_file) as writer:
        subset1.to_excel(writer,
                        'Criteria_1', index=False)
        subset2.to_excel(writer,
                    'Criteria_2', index=False)
        subset3.to_excel(writer,
                    'Criteria_3', index=False)
        subset4.to_excel(writer,
                    'Criteria_4', index=False)

In [8]:
contact_pull(file = 'BMI_Data_UPDATE.xlsx', 
             bmi_sheet = 'HeightWeight',
             contact_sheet = 'Contact Info',
             output_file = 'BMI_Solution_UPDATE.xlsx')

In [12]:
exec(open('automate_data_check_bmi.py').read())